In [12]:
from operator import itemgetter
from typing import List, Tuple

from fastapi import FastAPI
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, format_document
from langchain_core.runnables import RunnableMap, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langserve import add_routes
from langserve.pydantic_v1 import BaseModel, Field

In [13]:
from langchain_ollama import ChatOllama
model_name = "jcai/taide-lx-7b-chat"
model = ChatOllama(model=model_name)
model

# import os
# from langchain_openai import AzureChatOpenAI
# model = AzureChatOpenAI(
#     azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
#     azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
#     openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
# )

ChatOllama(model='jcai/taide-lx-7b-chat')

In [14]:
from langchain_ollama import OllamaEmbeddings
emb = OllamaEmbeddings(
    model=model_name,
)

# from langchain_openai import AzureOpenAIEmbeddings
# emb = AzureOpenAIEmbeddings(
#     model="text-embedding-ada-002",
#     azure_endpoint='https://lang-chain-dev.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15',
#     azure_deployment='text-embedding-ada-002',
#     openai_api_version='2023-05-15'
# )

In [15]:
# ! pip3 install pypdf
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('./data/台灣人壽金美鑫美元利率變動型終身壽險.pdf')
docs = loader.load()
docs

[Document(metadata={'source': './data/台灣人壽金美鑫美元利率變動型終身壽險.pdf', 'page': 0}, page_content='第 1 頁，共  3 頁 台灣人壽金美鑫 美元利率變動型 終身壽險  \n保險商品內容說明  \n【承保範圍】  \n【增值回饋分享金的給付及通知】  \n第十二條 \n本公司於本契約有效期間內之每一保單年度屆滿後，除被保險人保險年齡達 16歲前者，按第三項約\n定辦理外，將依要保人於投保時所選擇下列方式之一給付增值回饋分享金：  \n一、  購買增額繳清保險金額：選擇購買增額繳清保險金額者，以增值回饋分享金為躉繳純保險費，計\n算自該保單週年日當日起生效之增額繳清保險金額，但被保險人為 受監護宣告尚未撤銷者 ，應依\n保單條款 第十五條約定辦理。  \n二、  現金給付：選擇現金給付者， 依本契約約定以現金給付 增值回饋分享金予要保人，惟須於第六保\n單年度屆滿後之每一保單週年日起，始得依本款方式給付。  \n三、  儲存生息： 選擇儲存生息者，各年度之增值回饋分享金將按各保單週年日當月之宣告利率依據年\n複利方式，累積至要保人請求時給付，或至被保險人身故、完全失能或本契約終止時，由本公司\n主動一併給付。但在本公司給付受益人保險金而終止契約的情形，要保人未請求之增值回饋分享\n金及其孳息，由該保險金受益人受領。惟須於第六保單年度屆滿後之每一保單週年日起，始得依\n本款方式給付。  \n要保人若未選擇者，則視為選擇 購買增額繳清保險金額 ，並得於本契約有效期間 內，以書面通知本公\n司變更前項給付方式，惟第六保單年度屆滿前，增值回饋分享金限以購買增額繳清保險金額辦理。  \n被保險人保險年齡到達 16歲前，其增值回饋分享金採儲存生息方式辦理，並應於被保險人保險年齡\n到達16歲時，就累計儲存生息之金額一次購買增額繳清保險金額，其後保單年度適用第一項規定。  \n要保人終止本契約，或被保險人於保險年齡達 16歲前死亡或致成完全失能程度者，本公司應退還歷\n年累計儲存生息之金額予要保人。  \n本公司於每一保單年度屆滿後，應將該增值回饋分享金之金額，以書面或電子郵件方式通知要保人。  \n \n【身故保險金或喪 葬費用保險金的給付】  \n第十五條 \n被保險人於本契約有效期間內身故者，本公司

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separators=["\n\n", "，", "。", "【"])

splits = text_splitter.split_documents(docs)
len(splits)

5

In [17]:
splits[1].page_content # Document

'，應將該增值回饋分享金之金額，以書面或電子郵件方式通知要保人。  \n \n【身故保險金或喪 葬費用保險金的給付】  \n第十五條 \n被保險人於本契約有效期間內身故者，本公司按下列三款取其最大值給付身故保險金：  \n一、身故日之當年度保險金額。  \n二、身故日之保單價值準備金 乘以保單價值準備金比率所得之金額 。 \n三、身故日之應繳保險費總和。  \n訂立本契約時，以未滿 15足歲之未成年人為被保險人，除喪葬費用之給付外，其餘死亡給付之約定\n於被保險人滿 15足歲之日起發生效力；被保險人滿 15足歲前死亡者，其身故保險金變更為喪葬費用\n保險金。  \n前項未滿 15足歲之被保險人如有於民國九十九年二月三日 (不含)前訂立之保險契約，其喪葬費用保\n險金之給付依下列方式辦理：  \n一、被保險人於民國九十九年二月三日 (不含)前訂立之保險契約，喪葬費用保險金額大於或等於遺產\n及贈與稅法第十七條有關遺產稅喪葬費扣除額之半數 (含)者，其喪葬費用保險金之給付，從其約\n定，一百零九年六月十二日 (含)以後所投保之喪葬費用保險金額，本公司不負給付責任，並應無\n息退還該超過部分之已繳保險費。  \n二、被保險人於民國九十九年二月三日 (不含)前訂立之保險契約，喪葬費用保險金額小於遺產及贈與\n稅法第十七條有關遺產稅喪葬費扣除額之半數 (含)者應加計民國一百零九年六月十二日 (含)以\n後所投保之喪葬費 用保險金額，被保險人死亡時，受益人得領取之喪葬費用保險金總和（不限本'

In [18]:
vectorstore.delete_collection()

In [19]:
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(documents=splits, embedding=emb)


KeyboardInterrupt: 

In [7]:
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.5}
)

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain

### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    model, retriever, contextualize_q_prompt
)


from langchain.chains.combine_documents import create_stuff_documents_chain
### Answer question ###
system_prompt = (
    "你是一個有用的助手, 你的任務是回答問題."
    "你必須根據以下提供的檢索內容進行問答問題."
    "如果檢索內容為空, 則回答 '沒有找到相關資訊'"
    "以 5 至 10 句話以內回應, 保持答案的簡潔"
    "以下為檢索內容:\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(model, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [9]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

### Statefully manage chat history ###
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [10]:

for r in conversational_rag_chain.stream(
    {"input": "被保險人未滿15歲死亡喪葬費用怎麼賠?"},
    config={"configurable": {"session_id": "abc122"}},
):
    if 'answer' in r:
        print(r['answer'], end="")
    elif 'context' in r:
        ## RAG 
        for doc in r['context']:
            print(f'\t文件: {doc.metadata['source']}')
            print(f'\t內文: {doc.page_content[:50]}')
        print('----')
    else:
        print(r)
        print('----')

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


{'input': '被保險人未滿15歲死亡喪葬費用怎麼賠?', 'chat_history': []}
----
	文件: ./data/台灣人壽金美鑫美元利率變動型終身壽險.pdf
	內文: ，應將該增值回饋分享金之金額，以書面或電子郵件方式通知要保人。  
 
【身故保險金或喪 葬費用保險
	文件: ./data/台灣人壽金美鑫美元利率變動型終身壽險.pdf
	內文: 第 2 頁，共  3 頁 公司） ，不得超過遺產及贈與稅法第十七條有關遺產稅喪葬費扣除額之半數。超過
	文件: ./data/台灣人壽金美鑫美元利率變動型終身壽險.pdf
	內文: 第 3 頁，共  3 頁 本公司依 保單條款 第十六條約定計算之完全失能保險金扣除被保險人之指定保險
	文件: ./data/台灣人壽金美鑫美元利率變動型終身壽險.pdf
	內文: ，本公司按下列三款取其
最大值給付完全失能保險金：  
一、完全失能診斷確定日之當年度保險金額。  
----
根據提供的資料，被保險人未滿15歲死亡時，身故保險金變更為喪葬費用保險金。具體賠付方式如下：

1. 如果保險契約是在民國99年2月3日（不含）前訂立且喪葬費用保險金超過遺產及贈與稅法第十七條有關遺產稅喪葬費扣除額的半數，則按照約定給付，但109年6月12日（含）以後投保的部分不負給付責任，並無息退還超過部分的已繳保險費。

2. 如果保險契約是在民國99年2月3日（不含）前訂立且喪葬費用保險金低於遺產及贈與稅法第十七條有關遺產稅喪葬費扣除額的半數，則需要加計109年6月12日（含）以後投保的部分，總和不得超過遺產稅喪葬費扣除額的半數，超過部分不負給付責任，並無息退還超過部分的已繳保險費。

3. 以受監護宣告尚未撤銷者為被保險人，其身故保險金變更為喪葬費用保險金。

4. 若多家保險公司投保，總額超過上述限額，則各保險公司按比例分擔至喪葬費用額度上限。